## Monopoly Game Agent (v3)

The following notebook creates agents to automatically play Monopoly. You can
specify the behaviour of these agents in two ways:
- How the agents decide whether to buy a property
- How the agents decide whether to build a house/hotel on a property

At present, more extensive decision-making features (like trading or mortgaging) are not included. However, these could be added in future versions.

### Project Setup (Do not edit)

Install dependencies

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Import dependencies

In [2]:
# monopoly simulator
from simulator.monosim.player import Player
from simulator.monosim.board import get_board, get_roads, get_properties, get_community_chest_cards, get_bank

# LangChain
from langchain_openai import ChatOpenAI
from langchain import OpenAI, LLMChain, PromptTemplate

# other imports
from pydantic import BaseModel, Field
from typing import List

# dotenv for loading API keys
import os
from dotenv import load_dotenv

#Python 3.9.6
from typing import Union

In [3]:
# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

### Courtroom Architecture
The following three chains will generate arguments for, against, and a final decision.

In [43]:
# Lawyer Pro: Argument for buying the property
pro_prompt = PromptTemplate(
    input_variables=["property_details", "player_stats"], # can prolly add past_cases here
    template="""
    You are a lawyer arguing FOR the decision to buy a property in Monopoly.

    Property Details:
    {property_details}

    Player Stats:
    {player_stats}

    Build your argument FOR why buying this property is a strategic decision to maximize winning chances.
    """
    # Can add the following later.
    # Past Relevant Cases:
    # {past_cases}
)
pro_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=pro_prompt)

# Lawyer Con: Argument against buying the property
con_prompt = PromptTemplate(
    input_variables=["property_details", "player_stats"], # can prolly add past_cases here
    template="""
    You are a lawyer arguing AGAINST the decision to buy a property in Monopoly.

    Property Details:
    {property_details}

    Player Stats:
    {player_stats}

    Build your argument AGAINST why buying this property is a strategic decision.

    Can add the following later.
    Past Relevant Cases:
    
    {past_cases}
    """
)
con_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=con_prompt)

# Judge: Final decision based on arguments
judge_prompt = PromptTemplate(
    input_variables=["property_details", "player_stats", "arguments_for", "arguments_against"],
    template="""
    You are a judge deciding whether a player should buy a property in Monopoly. Your goal is to maximize the player's chances of winning the game with forward-thinking reasoning.
    
    Property Details:
    {property_details}

    Player Stats:
    {player_stats}

    Argument FOR buying the property:
    {arguments_for}

    Argument AGAINST buying the property:
    {arguments_against}

    Make your final decision: Should the player buy this property? Justify your reasoning briefly.
    Output "buy" or "not buy" as your final decision.
    """
)
judge_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini"), prompt=judge_prompt)


### Reflection Vectorstore (unfinished, @Benny)
### LMAOO
The following will be used to store and retrieve past cases from the agents.

In [44]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

vectorstore_path = "vectorstore"

embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

try:
    vectorstore = FAISS.load_local(vectorstore_path, embedding_model, allow_dangerous_deserialization=True)

except RuntimeError:
    print("Vectorstore not found. Initializing a new one.")

    dummy_doc = [Document(page_content="Temporary initialization document.")]
    
    vectorstore = FAISS.from_documents(dummy_doc, embedding_model)  

    vectorstore.save_local(vectorstore_path)  


def retrieve_similar_cases(context: str, k: int = 3):
    """
    Retrieve similar cases from the vectorstore based on the provided context.
    """
    results = vectorstore.similarity_search(context, k=k)
    return results if results else []

def add_case_to_vectorstore(case_id: str, context: str, arguments: str, decision: str):
    """
    Add a new case to the vectorstore for future retrieval.
    """
    new_case = Document(
        page_content=f"Context: {context}\nArguments: {arguments}\nDecision: {decision}",
        metadata={"case_id": case_id}
    )
    vectorstore.add_documents([new_case])
    vectorstore.save_local(vectorstore_path)  # Save updates locally


### Buying Properties (Edit here)

This function can be modified to get the agent to behave differently when deciding whether to buy a property. The placeholders are just examples of what you can do. Note: you **don't need to use all the information available when making a decision**.

In [45]:
def custom_buy(self, dict_property_info : dict) -> Union[str, None]:
    """
    Edit this function to determine whether the player should buy a property. 

    Return:
    YOUR FUNCTION MUST RETURN "buy" (type str) to buy the property 
    or None (type None) to not buy the property.
    
    WARNING: I've included exhaustive documentation on all the data you can 
    use to make this decision. DON'T use all of it; it's mostly irrelevant. 

    Parameters:
    self: useful for accessing the following methods
    - self.get_state(): returns player data in a dict with the keys
        - cash: the amount of cash the player has
        - owned_roads: a list of owned roads
        - owned_stations: a list of owned stations
        - owned_utilities: a list of owned utilities
        - owned_colors: a dict which maps colors to True if the player owns all 
            properties of that color. Ex: owned_colors['blue'] = True/False
        - owned_houses_hotels: a dict which maps property names to the number of 
            houses/hotels built there. Ex: owned_houses_hotels['Park Place'] = 3
        - mortgageable_amount: total mortgageable value of player's properties
        WARNING: PROBABLY NOT USEFUL DATA BELOW
        - morgaged_roads: a list of mortgaged roads
        - mortgaged_stations: a list of mortgaged stations
        - mortgaged_utilities: a list of mortgaged utilities
        - name: name of player
        - number: player number
        - position: player's position on the board
        - dice_value: the sum of the dice rolled
        - jail_count: the number of turns the player has been in jail
        - exit_jail: whether the player has exited jail
        - free_visit: whether the player is visiting jail for free
        - has_lost: whether the player has lost the game
    - self._dict_properties[property_name]: returns property info for an 
    arbitrary property with a dict like dict_property_info below

    dict_property_info: Has a dict with info about the property to buy/not buy. 
        The dict has the following keys:
    * name: the name of the property
    * belongs_to: the player who owns the property
    * price: the price of the property
    * rent: the rent of the property
    - color: the color of the property
    - rent_with_color_set: rent when all properties of same color are owned
    * type: the type of property (one of 'road', 'station', 'utility')
    - houses_cost: the cost of building a house
    - hotels_cost: the cost of building a hotel
    - rent_with_X_houses_0_hotels: rent when X = {1, 2, 3, 4} houses are built
    - rent_with_4_houses_1_hotels: rent when 4 houses and 1 hotel are built
    WARNING: PROBABLY NOT USEFUL DATA BELOW
    * mortgage_value: the amount of money earned when mortgaging the property
    * unmortgage_value: the amount of money cost to unmortgage the property
    * is_mortgaged: whether the property is mortgaged
    * board_num: the position of the property on the board
    P.S. (* means for all property types, - means for road properties only)
    """

    # currently, no trading. So don't buy property if someone else owns it
    if dict_property_info['belongs_to'] != None:
        return None

    # If the property isn't owned, I get the LLM to decide whether to buy it
    # Steps: 1. making a prompt template, 2. injecting data, 3. setting up 
    # output parsing, and 4. running an LLM.

    ##################################################
    # Step 1: Gather game state and property data
    ##################################################
    property_details = f"""
    Property Name: {dict_property_info['name']}
    Type: {dict_property_info['type']}
    Cost: {dict_property_info['price']}
    Base Rent: {dict_property_info['rent']}
    """



    # Add specific details for each property type
    if dict_property_info['type'] == 'road':
        # Calculate number of properties of the same color owned
        n_color_properties = 0
        for property_name in self.get_state()['owned_roads']:
            property_name = property_name.strip().lower()  # normalize name
            prop_info = self._dict_properties.get(property_name)  # safe access
            if prop_info and prop_info['color'] == dict_property_info['color']:
                n_color_properties += 1

        property_details += f"""
        Color: {dict_property_info['color']}
        Cost of Building a House: {dict_property_info['houses_cost']}
        Cost of Building a Hotel: {dict_property_info['hotels_cost']}
        Rent with 4 Houses and 1 Hotel: {dict_property_info['rent_with_4_houses_1_hotels']}
        Number of {dict_property_info['color']} properties owned: {n_color_properties}
        """
    elif dict_property_info['type'] == 'station':
        property_details += f"""
        Number of Stations Owned: {len(self.get_state()['owned_stations'])}
        """
    elif dict_property_info['type'] == 'utility':
        property_details += f"""
        Number of Utilities Owned: {len(self.get_state()['owned_utilities'])}
        """

    player_stats = f"""
    Cash: {self.get_state()['cash']}
    Owned Roads: {len(self.get_state()['owned_roads'])}
    Owned Stations: {len(self.get_state()['owned_stations'])}
    Owned Utilities: {len(self.get_state()['owned_utilities'])}
    """

    # ##################################################
    # # Step 2: Retrieve past cases
    # ##################################################
    past_cases = retrieve_similar_cases(context=f"{property_details} {player_stats}", k=3)
    past_cases_text = "\n".join([case.page_content for case in past_cases])

    ##################################################
    # Step 3: Lawyer Pro and Lawyer Con Arguments
    ##################################################
    arguments_for = pro_chain.run(
        property_details=property_details,
        player_stats=player_stats,
        past_cases=past_cases_text
    )

    arguments_against = con_chain.run(
        property_details=property_details,
        player_stats=player_stats,
        past_cases=past_cases_text
    )

    ##################################################
    # Step 4: Judge's Decision
    ##################################################
    decision = judge_chain.run(
        property_details=property_details,
        player_stats=player_stats,
        arguments_for=arguments_for,
        arguments_against=arguments_against
    )

    # ##################################################
    # # Step 5: Store the decision in the vectorstore
    # ##################################################
    add_case_to_vectorstore(
        case_id=f"{dict_property_info['name']}_{self.get_state()['name']}",
        context=f"{property_details} {player_stats}",
        arguments=f"For: {arguments_for}\nAgainst: {arguments_against}",
        decision=decision
    )

    ##################################################
    # Step 6: Return Decision
    ##################################################
    # for us to see the output
    print(f"Lawyer FOR Arguments: {arguments_for}\n\n")
    print(f"Laywer AGAINST Arguments: {arguments_against}\n\n")
    print(f"Judge's Decision: {decision}\n\n\n\n")
    return "buy" if "buy" in decision.lower() else None

    # ##################################################
    # # Step 1: Making a prompt template
    # ##################################################

    # template = """
    # You are a strategic decision-maker playing a game of {game}.

    # Rules of the game:
    # {rules} 
    
    # Your goal:
    # Maximise your chances of winning {game} with forward-thinking reasoning. You want to have the most money at the end of the game.

    # Your task:
    # Analyse whether to buy {property_name} given the data that follows. Consider how you could make the other players go bankrupt.

    # Property Information:
    # - Cost: {price}
    # - Base rent: {rent}
    # """

    # # Only add color data when available (for roads)
    # if (dict_property_info['type'] == 'road'):
    #     template += "\n- Cost of building a house: {houses_cost}\n- Cost of building a hotel: {hotels_cost}\n- Rent with 4 houses and a hotel: {rent_with_houses_and_hotel}\n- Color: {color}\n- Number of {color} properties you own: {n_color_properties}\n\nYou currently have {cash} in cash and {n_roads} roads."
    # elif (dict_property_info['type'] == 'station'):
    #     template += "\n\nYou currently have {cash} in cash and {n_stations} stations."
    # elif (dict_property_info['type'] == 'utility'):
    #     template += "\n\nYou currently have {cash} in cash and {n_utilities} utilities."
    # template += "\nDetermine whether you should buy {property_name}. Then output yes/no as your final answer."

    
    # ##################################################
    # # Step 2: Injecting data
    # ##################################################

    # inject = {
    #     'agent_name': self.get_state()['name'],
    #     'game': 'Monopoly',
    #     # Insert better rules later
    #     'rules': 'Typical Monopoly rules, but you cannot trade with other players or mortgage properties.',
    #     'property_name': dict_property_info['name'],
    #     'price': dict_property_info['price'],
    #     'rent': dict_property_info['rent'],
    #     'cash': self.get_state()['cash'],
    #     'n_roads': len(self.get_state()['owned_roads']),
    #     'n_stations': len(self.get_state()['owned_stations']),
    #     'n_utilities': len(self.get_state()['owned_utilities']),
    #     'houses_cost': dict_property_info['houses_cost'],
    #     'hotels_cost': dict_property_info['hotels_cost'],
    #     'rent_with_houses_and_hotel': dict_property_info['rent_with_4_houses_1_hotels']
    # }

    # # Only add color data when available (for roads)
    # if (dict_property_info['type'] == 'road'):
    #     inject['color'] = dict_property_info['color']
        
    #     # Bugs here! self._dict_properties.get(property) fails for some
    #     # properties. Don't reuse this code!
    #     n_color_properties = 0
    #     for property in self.get_state()['owned_roads']:
    #         prop_info = self._dict_properties.get(property)
    #         if prop_info and prop_info['color'] == dict_property_info['color']:
    #             n_color_properties += 1
    #     inject['n_color_properties'] = n_color_properties

    # prompt_template = PromptTemplate(input_variables=inject.keys(), template=template)
    # prompt = prompt_template.format(**inject)

    
    # ##################################################
    # # Step 3: Setting up output parsing
    # ##################################################

    # class Output(BaseModel):
    #     reasoning: str = Field(description="Your reasoning for the decision")
    #     decision: str = Field(description="Whether to buy the property (yes/no)")

    # model = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)
    # structured_llm = model.with_structured_output(Output)
    
    
    # ##################################################
    # # Step 4: Running the LLM
    # ##################################################
    
    # response = structured_llm.invoke(prompt)
    # print(response)
    
    # return "buy" if response.decision == "yes" else None

### Backend Setup (Do not edit)

These are functions that setup a game and let you access the state of the game.

In [46]:
def modify_buy_or_bid(buy) -> str:
    return custom_buy

class CustomPlayer(Player):
    buy_or_bid = modify_buy_or_bid(Player.buy_or_bid)

In [47]:
def initialize_game() -> dict:
    """
    Initializes a game with two players and sets up the bank, board, roads, properties, 
    and community chest cards.
    
    Returns:
        dict: A dictionary containing the following:
            - "bank": Game's bank object.
            - "board": Main game board.
            - "roads": List of road objects.
            - "properties": List of property objects.
            - "community_chest_cards": Dictionary of community chest cards.
            - "players": List of two Player objects, with Player 1 first.
    """
    
    bank = get_bank()
    board = get_board()
    roads = get_roads()
    properties = get_properties()
    community_chest_cards = get_community_chest_cards()
    community_cards_deck = list(community_chest_cards.keys())

    # Note how we have one of our players vs. a default player that just buys 
    # whenever cash is available. We can change this later
    player1 = CustomPlayer('Alice', 1, bank, board, roads, properties, community_cards_deck)
    player2 = Player('Bob', 2, bank, board, roads, properties, community_cards_deck)
    
    player1.meet_other_players([player2])
    player2.meet_other_players([player1])
    
    return {
        "bank": bank,
        "board": board,
        "roads": roads,
        "properties": properties,
        "community_chest_cards": community_chest_cards,
        "players": [player1, player2]
    }

In [48]:
def get_current_state(players) -> dict:
    """
    Retrieves the current state of each player, including position, owned roads, 
    money, mortgaged properties, and other status details.

    Args:
        players (list[Player]): List of Player objects in the game.

    Returns:
        dict: A dictionary containing:
            - "players": A list of dictionaries, each with a player's state.
    """
    
    current_state = {
        "players": [{"state": player.get_state()} for player in players]
    }
    return current_state

### Run a Game (Be careful - it costs real money!)

In [49]:
game = initialize_game()

In [50]:
player1 = game["players"][0]
player2 = game["players"][1]
list_players = [player1, player2]

# WARNING: KEEP THIS UNDER 10 ROUNDS FOR NOW
# EACH ROUND COSTS US MONEY
STOP_AT_ROUND = 2

In [51]:
idx_count = 0
while not player1.has_lost() and not player2.has_lost() and idx_count < STOP_AT_ROUND:
    for player in list_players:
        # Uncomment this line to run the game if you're sure there are no bugs
        player.play() 
    idx_count += 1

Lawyer FOR Arguments: Ladies and gentlemen of the jury, let me present to you a compelling case for the strategic acquisition of Pall Mall in our game of Monopoly. The decision to invest in this property is not just a financial transaction; it is a carefully considered step towards enhancing our position in the game and maximizing our chances of victory. Allow me to outline the key reasons to support this argument.

### Strategic Positioning
First and foremost, Pall Mall is part of the purple property group, which historically offers excellent returns on investment. The cost to acquire Pall Mall is merely $140, a small fraction of our current cash reserves of $1500. By purchasing this property, we initiate our journey towards monopolizing a color set, which serves as a foundation for building houses and hotels – potential income generators that can transform our finances from passive to active income.

### Increasing Revenue Potential
Let’s consider the financial dynamics at play. The 

In [52]:
# === Test: Verify that game decisions were stored ===
print("\n🔍 Running post-game vector store verification...")

# Try retrieving past game cases (use any property from the game)
test_context = "Baltic Avenue costs $60. Player has $300 cash."
retrieved_cases = retrieve_similar_cases(test_context, k=3)

print(f"\n🧠 Retrieved {len(retrieved_cases)} case(s) from vector store after gameplay:")
for case in retrieved_cases:
    print(case.page_content)

# Ensure at least one new case was stored
assert len(retrieved_cases) > 0, "❌ No new cases stored after gameplay! FAISS is not saving correctly."
print("✅ Post-game vector store test passed.\n")



🔍 Running post-game vector store verification...

🧠 Retrieved 3 case(s) from vector store after gameplay:
Context: Boardwalk costs $400. Player has $600 cash.
Arguments: FOR: It's a high-value property. AGAINST: It costs too much.
Decision: buy
Context: 
    Property Name: bow street
    Type: road
    Cost: 180
    Base Rent: 14
    
        Color: orange
        Cost of Building a House: 100
        Cost of Building a Hotel: 100
        Rent with 4 Houses and 1 Hotel: 950
        Number of orange properties owned: 0
         
    Cash: 1406
    Owned Roads: 1
    Owned Stations: 0
    Owned Utilities: 0
    
Arguments: For: Ladies and gentlemen of the jury, today I present to you a compelling argument in favor of the decision to purchase Bow Street in the game of Monopoly. This opportunity, priced at a mere $180, is not just an addition to our collection of assets; it represents a strategic move that can significantly enhance our chances of winning in the long run. Allow me to elabo